In [32]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import scipy.ndimage as nd
import scipy

from skimage.feature import hog
from skimage import data, color, exposure
from sklearn.cluster import KMeans

import random

In [33]:
vocabulary = np.load('/oasis/projects/nsf/csd395/ruogu/vocabulary/vocabulary.npy');
M = vocabulary.shape[0];
L = 3;
saveDir = '/oasis/projects/nsf/csd395/ruogu/histogram/';
dataDir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/patches/';

In [34]:
kmeans = KMeans(init='random', n_clusters=M, n_init=10);
kmeans.fit(vocabulary);

In [41]:
# Return the histogram of an image by applying SIFT + K-means + SPM
# image: the input grayscaled image
# kmeans: python kmeans object used to predict
#             feature type of a new sift descriptor
# M: number of channels (feature typs)
# L: number of SPM levels.
def getHistogram(image, kmeans, M, L):
    x_size = image.shape[0];
    y_size = image.shape[1];
    # Get SIFT key points and descriptor.
    sift = cv2.SIFT();
    key_point, descriptor = sift.detectAndCompute(image, None);
    
    # Map key points with type.
    if descriptor is None:
        return None;
    cluster_labels = kmeans.predict(descriptor);
    
    # Level 0
    histogram = np.zeros(M);
    weight_0 = 1.0 / (2**L);
    for k in range(0, len(key_point)):
        histogram[cluster_labels[k]] += 1;
    histogram = histogram * weight_0;
    # Other levels
    for l in range(1, L):
        grid_x = 1.0 * x_size / (2**l);
        grid_y = 1.0 * y_size / (2**l);
        weight = 1.0 / (2**(L - l + 1));
        for i in range(0, (2**l)):
            x_min = i * grid_x;
            x_max = x_min + grid_x;
            for j in range(0, (2**l)):
                y_min = j * grid_y;
                y_max = y_min + grid_y;
                histogram_tmp = np.zeros(M);
                # Count and update histogram from this grid.
                for k in range(0, len(key_point)):
                    if (x_min <= key_point[k].pt[0] <= x_max and
                        y_min <= key_point[k].pt[1] <= y_max):
                        histogram_tmp[cluster_labels[k]] += 1;
                histogram_tmp = histogram_tmp * weight;
                histogram = np.hstack((histogram, histogram_tmp));
    return (histogram / len(key_point)); # Normalization

In [42]:
fileNames = os.listdir(dataDir);
for fileName in fileNames:
    # print fileName;
    histograms = [];
    images = np.load(dataDir + fileName);
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);
        h = (img, kmeans, M, L);
        if h is None:
            continue;
        else:
            histograms.append(h);
    np.save(saveDir + fileName[7:], histogram);